In [4]:
import subprocess
import os
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

# --- 1. 配置 ---
SOURCE_DOC_FILE = "./yq2021-0602文娱产业正离文化越来越远.doc"

# Milvus 配置
MILVUS_HOST = "192.168.16.138"
MILVUS_PORT = "19530"
COLLECTION_NAME = "report_analysis"
ID_FIELD = "chunk_id"
VECTOR_FIELD = "vector"
TEXT_FIELD = "text_content"
MODEL_NAME = 'all-mpnet-base-v2'
EMBEDDING_DIM = 768

# --- 2. 文档加载函数 (保持不变) ---
def find_libreoffice():
    possible_paths = [
        r"C:\Program Files\LibreOffice\program\soffice.exe",
        r"C:\Program Files (x86)\LibreOffice\program\soffice.exe",
        r"D:\LibreOffice\program\soffice.exe",
        "soffice"
    ]
    for path in possible_paths:
        try:
            if path != "soffice" and not os.path.exists(path): continue
            result = subprocess.run([path, '--version'], capture_output=True, text=True, timeout=10)
            if result.returncode == 0: return path
        except: continue
    return None

def load_doc_as_text(doc_path):
    # ... (与之前的加载逻辑相同，为节省空间省略重复代码，核心是转HTML提取文本) ...
    # 这里为了完整性，简写核心逻辑：
    libreoffice_path = find_libreoffice()
    if not libreoffice_path: return None
    
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.dirname(doc_path)
    html_filename = os.path.basename(doc_path).rsplit('.', 1)[0] + '.html'
    html_path = os.path.join(output_dir, html_filename)
    
    if os.path.exists(html_path): os.remove(html_path)
    
    cmd = [libreoffice_path, '--headless', '--convert-to', 'html', '--outdir', output_dir, doc_path]
    subprocess.run(cmd, capture_output=True, text=True)
    
    if not os.path.exists(html_path): return None
    
    # 尝试读取 (优先utf-8, 其次gb18030)
    content = ""
    for enc in ['utf-8', 'gb18030']:
        try:
            with open(html_path, 'r', encoding=enc) as f:
                content = f.read()
                break
        except: continue
            
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()
    
    # 清理临时文件
    try: os.remove(html_path)
    except: pass
    
    return text

# --- 3. 新的通用切割函数 (核心修改) ---
def split_text_recursive(text, chunk_size=500, chunk_overlap=50):
    """
    通用递归切割：适用于任何文档。
    不依赖特定标题，而是利用标点符号和段落进行智能分割。
    """
    print(f"正在使用通用策略切割文本 (目标大小: {chunk_size} 字符)...")
    
    # 1. 基础清理
    text = re.sub(r"\"", "", text)
    text = re.sub(r"[\x00-\x1f\x7f-\x9f]", "", text) # 去除控制字符
    text = re.sub(r"\n\s*\n", "\n", text) # 合并空行
    
    # 2. 定义分隔符优先级 (从大到小)
    # 先按段落切，太长就按句号切，还太长就按逗号切
    separators = ["\n\n", "\n", "。", "！", "？", "；", "，", " "]
    
    chunks = []
    
    # 简单的递归切割实现 (无需安装 LangChain)
    def recursive_split(text, separators):
        final_chunks = []
        
        # 找到当前优先级最高的分隔符
        separator = separators[0]
        next_separators = separators[1:] if len(separators) > 1 else []
        
        # 如果没有分隔符了，或者文本已经够短，直接返回
        if not separator:
            return [text]
        
        # 按分隔符切割
        splits = text.split(separator)
        current_chunk = ""
        
        for split in splits:
            # 恢复分隔符 (除了空格和换行，其他的标点最好保留)
            if separator not in ["\n", "\n\n", " "]:
                split += separator
                
            # 如果当前块加上这一段还很小，就合并
            if len(current_chunk) + len(split) < chunk_size:
                current_chunk += split
            else:
                # 如果当前块已经有内容，先保存
                if current_chunk:
                    final_chunks.append(current_chunk)
                    current_chunk = ""
                
                # 如果新的一段本身就很大，需要递归继续切
                if len(split) > chunk_size and next_separators:
                    sub_chunks = recursive_split(split, next_separators)
                    final_chunks.extend(sub_chunks)
                else:
                    # 否则直接作为新的一块
                    current_chunk = split
        
        # 处理最后剩余的块
        if current_chunk:
            final_chunks.append(current_chunk)
            
        return final_chunks

    chunks = recursive_split(text, separators)
    
    # 过滤太短的块
    chunks = [c.strip() for c in chunks if len(c.strip()) > 20]
    
    print(f"切割完成，共得到 {len(chunks)} 个文本块。")
    return chunks

# --- 4. 主流程 ---
def main():
    # 1. 加载
    full_text = load_doc_as_text(SOURCE_DOC_FILE)
    if not full_text: 
        print("无法读取文件内容。")
        return

    print(f"文档加载成功，长度: {len(full_text)} 字符")
    print(f"文档预览: {full_text[:50]}...")

    # 2. 切割 (使用新的通用函数)
    chunks = split_text_recursive(full_text)
    
    if not chunks:
        print("切割结果为空，请检查文档内容。")
        return

    # 预览
    print("\n--- 块预览 ---")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i}: {chunk[:50]}...")

    # 3. 存入 Milvus (保持原有逻辑)
    print(f"\n正在加载嵌入模型 '{MODEL_NAME}'...")
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(chunks, show_progress_bar=True)
    
    print(f"正在连接 Milvus ({MILVUS_HOST})...")
    # 如果是本地文件模式请修改这里: connections.connect("default", uri="./milvus_demo.db")
    connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    
    if utility.has_collection(COLLECTION_NAME):
        utility.drop_collection(COLLECTION_NAME)
        
    fields = [
        FieldSchema(name=ID_FIELD, dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name=VECTOR_FIELD, dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema(name=TEXT_FIELD, dtype=DataType.VARCHAR, max_length=65535)
    ]
    collection = Collection(name=COLLECTION_NAME, schema=CollectionSchema(fields))
    
    #  插入数据
    print(f"正在向 Milvus 插入 {len(chunks)} 条数据...")
    collection.insert([embeddings, chunks])
    
    print("正在等待数据写入磁盘 (Flush)...")
    collection.flush() 

    #  创建索引
    print("正在创建索引...")
    collection.create_index(
        VECTOR_FIELD, 
        {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
    )
    
    #  加载并显示结果
    collection.load()
    
    print(f"\n成功存入 {collection.num_entities} 条数据到 Milvus！")

if __name__ == "__main__":
    main()

文档加载成功，长度: 5969 字符
文档预览: 



〖特别报告〗      政府左右房价？（下）









2021年06月02日福卡分...
正在使用通用策略切割文本 (目标大小: 500 字符)...
切割完成，共得到 13 个文本块。

--- 块预览 ---
Chunk 0: 〖特别报告〗      政府左右房价？（下）2021年06月02日福卡分析             ...
Chunk 1: 2020年全国规模以上文化及相关产业企业实现营业收入98514亿元，其中，互联网+其他信息服务、其他...
Chunk 2: “你我本无缘，全靠我花钱”也直接催生了诸多追星金融套路，最具争议的莫过于集资，据蓝鲸财经报道，截至4...

正在加载嵌入模型 'all-mpnet-base-v2'...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


正在连接 Milvus (192.168.16.138)...
正在向 Milvus 插入 13 条数据...
正在等待数据写入磁盘 (Flush)...
正在创建索引...

成功存入 13 条数据到 Milvus！
